# SAR-LRA Tool for Google Colaboratory

This notebook facilitates the acquisition of Sentinel-1 SAR composite imagery for the VV_VH combination, encompassing both ascending and descending orbits, which can then be utilized to deploy the models.

Without modifications the notebook will predict in Taiwan.

### IMPORTANT:
Select a runtime with large RAM (such as TPU), otherwise the processing will brake.

In [ ]:
# install some libraries

!pip install geedim
!pip install leafmap==0.22.0
!pip install geemap==0.24.1
!pip install -U geemap
!pip install earthengine-api

In [ ]:
from datetime import timedelta
import datetime
import leafmap          # Version 0.22.0
import geemap           # Version 0.24.1
import ee               # Version 0.1.358
from glob import glob
import numpy as np      # Version 1.25.0
import pandas as pd     # Version 2.0.3
import os

# Print versions
print("leafmap:", leafmap.__version__)
print("geemap:", geemap.__version__)
print("Earth Engine API:", ee.__version__)
print("numpy:", np.__version__)
print("pandas:", pd.__version__)

## 2. Autenticate and initialize the Google Earth Engine

In [ ]:
m = geemap.Map()

## 3. Define the Area of Interest

Designe a polygon that includes your area of interest (AOI). For this tutorial we pre defined the AOI for Taiwan.

In [ ]:
# Load the world countries dataset
worldcountries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
# Filter Taiwan
filterCountry = ee.Filter.eq('country_co', 'TW')
AoI = worldcountries.filter(filterCountry)

# Create a Map instance
Map = geemap.Map()

# Add the country border to the map
Map.addLayer(AoI, {}, 'Country Border')

# Define Sentinel-2 post-event parameters
start_date = '2024-04-27'
end_date = '2024-05-01'
cloud_percentage = 30

# Load Sentinel-2 imagery
sentinel_2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate(start_date, end_date) \
    .filterBounds(AoI) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage))

# Print the number of images found
print("Number of images:", sentinel_2.size().getInfo())

# Calculate median and clip to the AOI
sentinel_2 = sentinel_2.median().clip(AoI)
# Select bands for true color imagery
sentinel_2 = sentinel_2.select(['B2', 'B3', 'B4'])
# Define true color visualization parameters
trueColor_palette = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000
}
# Add true color Sentinel-2 imagery to the map
Map.addLayer(sentinel_2, trueColor_palette, "Sentinel-2 True Color")

# Center the map on the dataset
Map.centerObject(AoI, 5)

# Display the map
Map

## 4. Define temporal buffers and temporal stacks
The following dates are for the Sumatra earthquake on the 25th of February 2022. Change the dates according to the MLE you want to deploy the model for.

Taiwan earthquake: 2/4/2024

IMPORTANT: The landslide MUST be occurred in between the pre_end and post_start dates.

In [ ]:
# Convert the drawn geometry to an Earth Engine Geometry object
# geometry = ee.FeatureCollection(Map.draw_features) # for this tutorial we will use the box_coordinates defined in the following box.

pre_stack_end = '2024-04-02'
post_stack_start = '2024-04-02'

In [ ]:
### UNCOMMENT THIS TO DEFINE GEOMETRY FOR THE TAIWAN EVENT

box_coordinates = [[
    [121.28320081506229,23.673234207076405],
    [121.83526380334354,23.673234207076405],
    [121.83526380334354,24.460681113046533],
    [121.28320081506229,24.460681113046533],
    [121.28320081506229,23.673234207076405]
]]

geometry = ee.Geometry.Polygon(box_coordinates)

## 5. Process and Download Sentinel-1 SAR composite images
In certain areas where VH polarization data is unavailable, the notebook may encounter errors as it is designed to download both VV and VH data. The same applies to the ascending and descending orbits.

In [ ]:
print('SENTINEL 1 SAR IMAGE PROCESSING AND ACQUISITION !')
print('_____________________________________________________________________________________')

orbits = ['DESCENDING', 'ASCENDING'] # Orbits to download
pre_days = 60 # pre event stack dimensions
post_days = 12 # pre event stack dimensions

# Define date ranges
pre_end = datetime.datetime.strptime(pre_stack_end, "%Y-%m-%d")
pre_start = pre_end - datetime.timedelta(days=pre_days)
print('Pre stack start:', pre_start)
print('Pre stack end:',pre_end)
pre_end = ee.Date(pre_end)
pre_start = ee.Date(pre_start)

post_start = datetime.datetime.strptime(post_stack_start, "%Y-%m-%d")
post_end = post_start + datetime.timedelta(days=post_days)
print('Post stack start:',post_start)
print('Post stack end:',post_end)
post_start = ee.Date(post_start)
post_end = ee.Date(post_end)

for orbit in orbits:
    print('Orbit: ', orbit)
    project_path = ''
    inputs_path = os.path.join(*[project_path, f'deploy/VV_VH/60_{post_days}'])
    outputs_path = os.path.join(*[project_path, 'outputs'])
    print('project_path: ', project_path); print('training_path: ', inputs_path); print('outputs_path: ', outputs_path)

    print('_____________________________________________________________________________________')

    # Make Image Collections
    pre_data = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(geometry) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.eq('orbitProperties_pass', orbit)) \
        .filterDate(pre_start, pre_end)

    pre_count = pre_data.size().getInfo()
    print('Pre image count:', pre_count)

    # Get a list of images in the collection
    pre_list = pre_data.toList(pre_count)

    # Iterate through each image in the collection
    for i in range(pre_count):
        image = ee.Image(pre_list.get(i))
        image_id = image.id().getInfo()
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        print('Image ID:', image_id, ' Date:', date)

    post_data = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(geometry) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.eq('orbitProperties_pass', orbit)) \
        .filterDate(post_start, post_end)

    post_count = post_data.size().getInfo()
    print('Post image count:', post_count)

    # Get a list of images in the collection
    post_list = post_data.toList(post_count)

    # Iterate through each image in the collection
    for i in range(post_count):
        image = ee.Image(post_list.get(i))
        image_id = image.id().getInfo()
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        print('Image ID:', image_id, ' Date:', date)

    ### VV ###
    pre_corrected_masked_VV = pre_data.select('VV') # Select VV polarization
    pre_median_VV = pre_corrected_masked_VV.reduce(ee.Reducer.median())  # Calculate median
    pre_median_VV = pre_median_VV.select(0).rename('preVV')  # Rename bands


    post_corrected_masked_VV = post_data.select('VV')  # Select VV polarization
    post_median_VV = post_corrected_masked_VV.reduce(ee.Reducer.median())  # Calculate median
    post_median_VV = post_median_VV.select(0).rename('postVV')  # Rename bands

    diff_VV = post_median_VV.subtract(pre_median_VV).rename('diffVV') # Change detection (post event - pre event)

     ### VH ###
    pre_corrected_masked_VH = pre_data.select('VH')   # Select VV polarization
    pre_median_VH = pre_corrected_masked_VH.reduce(ee.Reducer.median())  # Calculate median
    pre_median_VH = pre_median_VH.select(0).rename('preVH')  # Rename bands


    post_corrected_masked_VH = post_data.select('VH')  # Select VV polarization
    post_median_VH = post_corrected_masked_VH.reduce(ee.Reducer.median())  # Calculate median
    post_median_VH = post_median_VH.select(0).rename('postVH')  # Rename bands

    diff_VH = post_median_VH.subtract(pre_median_VH).rename('diffVH') # Change detection (post event - pre event)

    DS = post_median_VV.addBands(post_median_VH).addBands(diff_VV).addBands(diff_VH) # Create composite SAR image
    DS = DS.clip(geometry) # Clip composite SAR image to Area of Interest

    ### GEE does not allow big downloads - therefore we dowload grid by grid, and re-merge afterwards.

    # create fishnet (sample grid)
    fishnet = geemap.fishnet(geometry, h_interval=0.5, v_interval=0.5, delta=1)

    # download composite tiles
    geemap.download_ee_image_tiles(DS, fishnet, inputs_path+'/'+'/VV_VH_'+orbit+'/', prefix="VV_VH_"+orbit+'_'+'_', scale=10, crs=ee.Projection('EPSG:4326'))

    # merge tiles and save
    leafmap.merge_rasters(inputs_path+'/'+'/VV_VH_'+orbit, output=inputs_path+'/'+'/SAR_'+orbit+'_'+'.tif', input_pattern='*.tif')

    # delete useless rasters inside folders
    s2s = glob(inputs_path+'/'+'/VV_VH_'+orbit+'/*.tif')

    for s2_file in s2s:
        os.remove(s2_file)
    os.rmdir(inputs_path+'/'+'/VV_VH_'+orbit)

print('DONE !!!')

# SAR-LRA VV_VH Models Deployment
The following part of the notebook deploys the two models on the composite SAR images previously acquired for ascending and descending orbits, VV_VH combination.

In [ ]:
!pip install rasterio
!pip install tensorflow opencv-python-headless
!pip install requests

In [ ]:
import requests
import rasterio
import tensorflow as tf
from tensorflow.keras.applications import imagenet_utils
# from PIL import Image
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.optimizers import Adam
import numpy as np
import time
import cv2
import os

print("Rasterio version:", rasterio.__version__) # Version 1.2.10
print("TensorFlow version:", tf.__version__)  # Version 2.10.0
print("NumPy version:", np.__version__)       # Version 1.21.5
print("OpenCV version:", cv2.__version__)     # Version 4.5.5

In [ ]:
# define some useful functions

def CNN(lr, loss, filtersFirstLayer, drop, input_size=(64, 64, 4)):
    inputs = Input(shape=input_size)
    conv1 = Conv2D(filtersFirstLayer, 3, padding='same', activation='relu')(inputs)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D()(conv1)

    conv2 = Conv2D(filtersFirstLayer, 3, padding='same', activation='relu')(pool1)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D()(conv2)

    conv3 = Conv2D(filtersFirstLayer, 3, padding='same', activation='relu')(pool2)
    conv3 = BatchNormalization()(conv3)

    target_shape = (conv3.shape[1], conv3.shape[2])

    resized_tensor_3 = tf.image.resize(conv3, target_shape)
    resized_tensor_2 = tf.image.resize(conv2, target_shape)
    resized_tensor_1 = tf.image.resize(conv1, target_shape)

    concatenated_tensor = tf.concat([resized_tensor_3, resized_tensor_3, resized_tensor_3], axis=-1)

    pool3 = MaxPooling2D()(concatenated_tensor)

    drop1 = Dropout(drop)(pool3)
    flat = Flatten()(drop1)
    en = Dense(filtersFirstLayer * 8, activation='relu')(flat)
    out = Dense(1, activation='sigmoid')(en)
    model = Model(inputs, out)

    model.compile(loss=loss, optimizer=Adam(learning_rate=lr), metrics='accuracy')
    return model

def focal_loss(y_true, y_pred, alpha=0.25, gamma=2.0):
    # Compute binary cross-entropy
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred, from_logits=True)

    # Compute the predicted probabilities for the true class
    p_t = tf.math.exp(-bce)

    # Compute the focal loss
    focal_loss = alpha * (1 - p_t) ** gamma * bce

    return focal_loss

def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

def sliding_window(image, step, ws):
    for y in range(0, image.shape[0] - ws[1], step):
        for x in range(0, image.shape[1] - ws[0], step):
            yield (x, y, image[y:y + ws[1], x:x + ws[0]])

## DESCENDING orbit

### 1. Defining useful variables
Here we define some variables and paths that will be useful to call model weights, images, and saving the outputs.

In [ ]:
place = 'Taiwan' # name of the event or location

orbit = 'DESCENDING'

# directory of the SAR composite image
image_path = f'deploy/VV_VH/60_12/SAR_{orbit}_.tif'

# ulr of the weights of the model
github_url = "https://github.com/lorenzonava96/SAR-and-DL-for-Landslide-Rapid-Assessment/raw/main/SAR-LRA%20Tool%20V1/model_weights/VV_VH_60_nn_noSlope_DESCENDING_60_12_5_size_64_filters_64_batch_size_512_lr_0.001_dropout_0.7_fil1_3_fil2_3_fil3_3.hdf5"

# size of the image
size = 64

# number of bands
channels = 4

### 2. Import Model and Image
Remember to define the model parameters as the one used in the training. We call the model weights from the github directory.


In [ ]:
# Define model parameters and import model
model = CNN(filtersFirstLayer=64, drop=0.7, lr=0.001, input_size=(size, size, channels), loss=focal_loss)
print("[INFO] loading model weights...")

# Make a GET request to fetch the contents of the HDF5 file
response = requests.get(github_url)

# Check if the request was successful
if response.status_code == 200:
    # Write the contents of the HDF5 file to a local file
    with open("model_weights_desce.hdf5", "wb") as file:
        file.write(response.content)
    print("HDF5 file downloaded successfully.")
else:
    print("Failed to fetch the HDF5 file from GitHub")

# Load the weights into the model
model.load_weights("model_weights_desce.hdf5")

# Load and preprocess the image
print("[INFO] loading image...")
with rasterio.open(image_path) as ori:
    tmp = np.moveaxis(ori.read(), 0, 2)
orig = np.asarray(tmp)
orig = orig[:, :, :(channels)]

print('MODEL AND IMAGE ARE READY !')

### 3. Run the Sliding Window algorithm
Here, we execute the sliding window algorithm to extract 64x64 images from the SAR composite image downloaded in the preceding notebook, 01_SAR-LRA_Sentinel-1_Image_Acquisition, with a specified overlap. Subsequently, we store these images along with their corresponding coordinates relative to the original image in two separate lists.

In [ ]:
rois = [] # list for images
locs = [] # list for coordinates

ROI_SIZE = (size, size) # 64x64
WIN_STEP = int(size/2)  # 32 to have 50% of overlap - modifiable

start = time.time()

for (x, y, roiOrig) in sliding_window(orig, WIN_STEP, ROI_SIZE):
    w = int(ROI_SIZE[0])
    h = int(ROI_SIZE[1])
    roi = cv2.resize(roiOrig, ROI_SIZE)
    roi = img_to_array(roi)
    rois.append(roi)
    locs.append((x, y, x + w, y + h))
    end = time.time()
print("[INFO] looping over pyramid/windows took {:.5f} seconds".format(end - start))

# convert the ROIs to a NumPy array
rois = np.array(rois)
print('[INFO] You extracted ', len(rois), 'patches')

print('ROIs ARE READY TO BE CLASSIFIED !')

### 4. Classify the extracted images
Here, we classify all the images (ROIs) extracted by the sliding window algorithm, saving the predictions as probabilities ranging from 0 to 1, indicating their likelihood of belonging to the landslide class. Subsequently, we will define a probability threshold to determine the class to which they belong.

In [ ]:
print("[INFO] classifying ROIs...")
start = time.time()
pred_datagen = ImageDataGenerator()
batch_size = 512
pred_ds = pred_datagen.flow(rois, batch_size = batch_size, seed = 42, shuffle=False)
ynew = model.predict(pred_ds) # predict
end = time.time()
print("[INFO] classifying ROIs took {:.5f} seconds".format(
    end - start))

print('THE MODEL CLASSIFIED ALL THE ROIs !')

## 5. Count the number of ROIs predicted as landslide


In [ ]:
n = 1
for i, prob in enumerate(ynew):
    if prob > 0.6: # probability threshold
        n += 1
print(n)

## 6. Append coordinates and probability value of the ROIs predicted as landslide

In [ ]:
L = []
P = []
for i, prob in enumerate(ynew):
    if prob > 0.6: # probability threshold
        box = locs[i]
        L.append(box)

## 7. Deploy Non-Maximum Suppression
The non max suppression used is the one developed by Adrian Rosebrock and it is very well explained here: https://pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/

In [ ]:
boxes = np.array(L) # to array
boxes = non_max_suppression_fast(boxes, overlapThresh=0.1) # select overlap threshold between bounding boxes

## 8. Draw the final bounding boxes and save as TIFF file
We delineate the ultimate bounding boxes around the detected landslides by the model, and we generate the final georeferenced raster, marking landslide boxes contours with 1 and non-landslide areas with 0.

In [ ]:
# clone the original image
clone = orig.copy()

# create an empty image (with zeros)
c = np.zeros((clone.shape[0], clone.shape[1]), dtype=np.uint8)

# iterate through the boxes and draw them on the image
for (startX, startY, endX, endY) in boxes:
    # Draw rectangle (bounding box) on the image
    cv2.rectangle(c, (startX, startY), (endX, endY), color=1, thickness=1)

In [ ]:
# Get the current working directory
current_directory = os.getcwd()

# Print the current directory
print("Current directory:", current_directory)

In [ ]:
predictions_folder = "predictions"

# Create the predictions folder if it doesn't exist
if not os.path.exists(predictions_folder):
    os.makedirs(predictions_folder)

pred_path = f'predictions/{place}_DESCENDING.tif' # directory and name of output TIFF file
ori =  rasterio.open(image_path)
c = np.squeeze(c)

with rasterio.Env():
    profile = ori.profile
    profile.update(
        dtype=rasterio.float32,
        count=1,
        width= c.shape[-1],
        height= c.shape[-2],
        compress='lzw')
    with rasterio.open(pred_path, 'w', **profile) as dst:
        dst.write(c.astype(rasterio.float32), 1)

print('PREDICTION SAVED AS TIFF !')

### 9. Save the predictions as a Shapefile
The predictions are saved in the folder: predictions.


In [ ]:
!pip install geopandas

In [ ]:
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape

# Open the georeferenced TIFF image
with rasterio.open(pred_path) as src:
    # Read the raster data as a numpy array
    image_array = src.read(1)

    # Get the transform (georeferencing information)
    transform = src.transform
    crs = src.crs

    # Generate shapes for areas where pixel values are equal to 1
    shapes = list(shapes(image_array, transform=transform))

# Filter shapes where pixel value is 1
valid_shapes = [s for s, v in shapes if v == 1]

# Convert valid shapes to GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=[shape(s) for s in valid_shapes], crs=crs)
shapefile_path = pred_path + ".shp"
# Save the GeoDataFrame as a shapefile
gdf.to_file(shapefile_path)

## ASCENDING
We will iterate through the methodology once more. The elements that will vary are the model, with the new one trained on Ascending data, and the composite SAR image, which will now be from the Ascending orbit.

### 1. Defining useful variables
Here we define some variables and paths that will be useful to call model weights, images, and saving the outputs.

In [ ]:
place = 'Taiwan' # name of the event or location

orbit = 'ASCENDING'

# directory of the SAR composite image
image_path = f'deploy/VV_VH/60_12/SAR_{orbit}_.tif'

# url of the weights of the model
github_url = "https://github.com/lorenzonava96/SAR-and-DL-for-Landslide-Rapid-Assessment/raw/main/SAR-LRA%20Tool%20V1/model_weights/VV_VH_60_nn_noSlope_ASCENDING_60_12_6_size_64_filters_32_batch_size_512_lr_0.001_dropout_0.7_fil1_3_fil2_3_fil3_3.hdf5"

# size of the image
size = 64

# number of bands
channels = 4

### 2. Import Model and Image
Remember to define the model parameters as the one used in the training.

In [ ]:
# Define model parameters and import model
model = CNN(filtersFirstLayer=32, drop=0.7, lr=0.001, input_size=(size, size, channels), loss=focal_loss)
print("[INFO] loading model weights...")

# Make a GET request to fetch the contents of the HDF5 file
response = requests.get(github_url)

# Check if the request was successful
if response.status_code == 200:
    # Write the contents of the HDF5 file to a local file
    with open("model_weights_asce.hdf5", "wb") as file:
        file.write(response.content)
    print("HDF5 file downloaded successfully.")
else:
    print("Failed to fetch the HDF5 file from GitHub")

# Load the weights into the model
model.load_weights("model_weights_asce.hdf5")

# Load and preprocess the image
print("[INFO] loading image...")
with rasterio.open(image_path) as ori:
    tmp = np.moveaxis(ori.read(), 0, 2)
orig = np.asarray(tmp)
orig = orig[:, :, :(channels)]

print('MODEL AND IMAGE ARE READY !')

### 3. Run the Sliding Window algorithm
Here, we execute the sliding window algorithm to extract 64x64 images from the SAR composite image downloaded in the preceding notebook, 01_SAR-LRA_Sentinel-1_Image_Acquisition, with a specified overlap. Subsequently, we store these images along with their corresponding coordinates relative to the original image in two separate lists.

In [ ]:
rois = [] # list for images
locs = [] # list for coordinates

ROI_SIZE = (size, size) # 64x64
WIN_STEP = int(size/2)  # 32 to have 50% of overlap - modifiable

start = time.time()

for (x, y, roiOrig) in sliding_window(orig, WIN_STEP, ROI_SIZE):
    w = int(ROI_SIZE[0])
    h = int(ROI_SIZE[1])
    roi = cv2.resize(roiOrig, ROI_SIZE)
    roi = img_to_array(roi)
    rois.append(roi)
    locs.append((x, y, x + w, y + h))
    end = time.time()
print("[INFO] looping over pyramid/windows took {:.5f} seconds".format(end - start))

# convert the ROIs to a NumPy array
rois = np.array(rois)
print('[INFO] You extracted ', len(rois), 'patches')

print('ROIs ARE READY TO BE CLASSIFIED !')

### 4. Classify the extracted images
Here, we classify all the images (ROIs) extracted by the sliding window algorithm, saving the predictions as probabilities ranging from 0 to 1, indicating their likelihood of belonging to the landslide class. Subsequently, we will define a probability threshold to determine the class to which they belong.

In [ ]:
print("[INFO] classifying ROIs...")
start = time.time()
pred_datagen = ImageDataGenerator()
batch_size = 512
pred_ds = pred_datagen.flow(rois, batch_size = batch_size, seed = 42, shuffle=False)
ynew = model.predict(pred_ds) # predict
end = time.time()
print("[INFO] classifying ROIs took {:.5f} seconds".format(
    end - start))

print('THE MODEL CLASSIFIED ALL THE ROIs !')

### 5. Count the number of ROIs predicted as landslide

In [ ]:
n = 1
for i, prob in enumerate(ynew):
    if prob > 0.6: # probability threshold
        n += 1
print(n)

### 6. Append coordinates and probability value of the ROIs predicted as landslide

In [ ]:
L = []
P = []
for i, prob in enumerate(ynew):
    if prob > 0.6: # probability threshold
        box = locs[i]
        L.append(box)

### 7. Deploy Non-Maximum Suppression
The non max suppression used is the one developed by Adrian Rosebrock and it is very well explained here:
https://pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/

In [ ]:
boxes = np.array(L)
boxes = non_max_suppression_fast(boxes, overlapThresh=0.1)

### 8. Draw the final bounding boxes and save as TIFF file
We delineate the ultimate bounding boxes around the detected landslides by the model, and we generate the final georeferenced raster, marking landslide boxes contours with 1 and non-landslide areas with 0.

In [ ]:
# clone the original image
clone = orig.copy()

# Create an empty image (black image with zeros)
c = np.zeros((clone.shape[0], clone.shape[1]), dtype=np.uint8)

# Iterate through the boxes and draw them on the image
for (startX, startY, endX, endY) in boxes:
    # Draw rectangle (bounding box) on the image
    cv2.rectangle(c, (startX, startY), (endX, endY), color=1, thickness=1)

In [ ]:
pred_path = f'predictions/{place}_ASCENDING.tif' # directory and name of output TIFF file
ori =  rasterio.open(image_path)
c = np.squeeze(c)

with rasterio.Env():
    profile = ori.profile
    profile.update(
        dtype=rasterio.float32,
        count=1,
        width= c.shape[-1],
        height= c.shape[-2],
        compress='lzw')
    with rasterio.open(pred_path, 'w', **profile) as dst:
        dst.write(c.astype(rasterio.float32), 1)

print('PREDICTION SAVED AS TIFF !')

### 9. Save the predictions as a Shapefile
The predictions are saved in the folder: predictions.

In [ ]:
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape

# Open the georeferenced TIFF image
with rasterio.open(pred_path) as src:
    # Read the raster data as a numpy array
    image_array = src.read(1)

    # Get the transform (georeferencing information)
    transform = src.transform
    crs = src.crs

    # Generate shapes for areas where pixel values are equal to 1
    shapes = list(shapes(image_array, transform=transform))

# Filter shapes where pixel value is 1
valid_shapes = [s for s, v in shapes if v == 1]

# Convert valid shapes to GeoDataFrame
gdf2 = gpd.GeoDataFrame(geometry=[shape(s) for s in valid_shapes], crs=crs)
shapefile_path = pred_path + ".shp"
# Save the GeoDataFrame as a shapefile
gdf2.to_file(shapefile_path)

## 10. Display the results

In [ ]:
# Close any previously opened map
Map.close()

# Create a new map instance
Map = geemap.Map()

# Define the time period and cloud percentage for Sentinel-2 imagery
start_date = '2024-04-27'
end_date = '2024-05-01'
cloud_percentage = 30

# Load Sentinel-2 imagery
sentinel_2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage))

print("Number of images:", sentinel_2.size().getInfo())

# Calculate median and clip to the geometry
sentinel_2 = sentinel_2.median().clip(geometry)
# Select bands for true color imagery
sentinel_2 = sentinel_2.select(['B2', 'B3', 'B4'])
# Define true color visualization parameters
trueColor_palette = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000  # Adjust this value based on the radiometric properties of your image
}

# Add Sentinel-2 true color imagery to the map
Map.addLayer(sentinel_2, trueColor_palette, "Sentinel-2 RGB")

# Add other layers like DS, if available
Map.addLayer(DS, {'bands': ['postVV'], 'min': [-28], 'max': [4], 'gamma': 0.65}, 'postVV', False)
Map.addLayer(DS, {'bands': ['postVH'], 'min': [-28], 'max': [4], 'gamma': 0.65}, 'postVH', False)
Map.addLayer(DS, {'bands': ['diffVV'], 'min': [-28], 'max': [4], 'gamma': 0.65}, 'diffVV', False)
Map.addLayer(DS, {'bands': ['diffVH'], 'min': [-28], 'max': [4], 'gamma': 0.65}, 'diffVH', False)

# Convert GeoDataFrames to GeoJSON for visualization
geojson = leafmap.gdf_to_geojson(gdf, epsg="4326")
geojson2 = leafmap.gdf_to_geojson(gdf2, epsg="4326")

# Define styles for GeoJSON layers
style1 = {
    "stroke": True,
    "color": "blue",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "blue",
    "fillOpacity": 1,
}
style2 = {
    "stroke": True,
    "color": "red",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "red",
    "fillOpacity": 1,
}
hover_style = {"fillOpacity": 1}

# Add GeoJSON layers to the map
Map.add_geojson(geojson, layer_name="DESCENDING predictions", style=style1, hover_style=hover_style)
Map.add_geojson(geojson2, layer_name="ASCENDING predictions", style=style2, hover_style=hover_style)

# Center the map on the geometry
Map.centerObject(geometry)

# Display the map
Map
